# Performance Report

Code taken from: https://nextjournal.com/gkoehler/pytorch-mnist

## Imports

In [1]:
from deepchecks.vision.base import VisionDataset
from torchvision import models
import torchvision
import torch

In [2]:
mnist_data = torchvision.datasets.MNIST('./mnist', download=True)

In [3]:
n_epochs = 1
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [4]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./mnist', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./mnist', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
class MNistNet(nn.Module):
    def __init__(self):
        super(MNistNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [7]:
model = MNistNet()
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

In [8]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [9]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(model.state_dict(), './results/model_states_dict.pth')
      torch.save(optimizer.state_dict(), './results/optimizer.pth')

In [10]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [11]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

C:\Users\nirhu\AppData\Local\Temp/ipykernel_38044/453714901.py:17: UserWarning:

Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

c:\users\nirhu\documents\deepchecks\mlchecks\venv_with_jupyter\lib\site-packages\torch\nn\_reduction.py:42: UserWarning:

size_average and reduce args will be deprecated, please use reduction='sum' instead.




Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.360446
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.364010
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.329987
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.249067
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.288059
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.218085
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.214215
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.111192
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.092593
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.989863
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.911151
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.659503
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.869479
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.660094
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.577335
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.550470
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.304469
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.408615
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.306857
Train Epoch: 1 [12160

In [12]:
torch.save(model, './results/model.pth')

In [13]:
model = torch.load('./results/model.pth')
model.eval()


MNistNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [14]:
test()

C:\Users\nirhu\AppData\Local\Temp/ipykernel_38044/453714901.py:17: UserWarning:

Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.




Test set: Avg. loss: 0.1995, Accuracy: 9387/10000 (94%)

